## Set file path and data prep

In [ ]:
import os

# Print the current working directory - youj want to be in data_wrangling_1
print(f"Current working directory: {os.getcwd()}")

# Optionally, change the working directory if needed
# os.chdir('/Users/christines/Desktop/workspace/Phaseout-dashboard/data_wrangling_1') 

Current working directory: /Users/christines/Desktop/workspace/Phaseout-dashboard/data_wrangling_1


In [5]:
#IMPORTANT!!: Set the year of the GCPT data####
current_year = 2024
#current_year = datetime.now().year

In [6]:
#Set data_wrangling_2/raw_data as the path to put the files

import pandas as pd
import numpy as np
from datetime import datetime
import json

# Define the correct path using absolute or relative paths
path_to_data_wrangling_2_folder = "../data_wrangling_2"

print(f"Checking public assets folder path: {repr(path_to_data_wrangling_2_folder)}")

if os.path.isdir(path_to_data_wrangling_2_folder):
    print("Public assets folder exists!")
    path_to_data_folder = os.path.join(path_to_data_wrangling_2_folder, "raw_data")
    
    # Check and create the data folder if necessary
    if not os.path.exists(path_to_data_folder):
        os.mkdir(path_to_data_folder)
        print(f"Creating data folder at: {path_to_data_folder}")
    else:
        print(f"Data folder already exists at: {path_to_data_folder}")
else:
    print(f"Public assets folder does not exist at: {path_to_data_wrangling_2_folder}")

Checking public assets folder path: '../data_wrangling_2'
Public assets folder exists!
Creating data folder at: ../data_wrangling_2/raw_data


In [7]:
#read in all the files needed

#read in latest gcpt
gcpt = pd.read_excel("data/July2024GCPT.xlsx")

#add capacity status data from other dashboard - or any list of region with total operating capacity
capacity_data = pd.read_excel('data/Capacity_status_H12024.xlsx')

# read in OECD data (year a country should phase out coal)
oecd_data = pd.read_excel('data/OECD.xlsx')

#read in planned coal phaseout year data
phaseout_data = pd.read_excel('data/Coal_phaseout_year.xlsx')

#add regions df
regions = pd.read_excel("data/GCPT Regions.xlsx")

In [8]:
#Join regions with gcpt
gcpt_regions = pd.merge(gcpt, regions, how="left", left_on="Country/Area", right_on="Country/Area")

#create data df that drops Country column from data and adds country/area from region data
data = gcpt_regions.copy()
data = data.drop('Country/Area', axis=1)

#replace and explode
data['Country/Area'] = data['Regions'].str.split(', ')
data = data.explode('Country/Area')
data = data[data["Country/Area"] != "No"]
data = data[data["Country/Area"] != "ROTW"]

#turn capacity from MW to GW, reset index
data['Capacity (MW)'] = data['Capacity (MW)']/1000
data.rename({"Capacity (MW)": "Capacity"}, axis=1, inplace=True)
data = data.reset_index(drop=True)
#data.head()

#create df that filters out proposed and retired plants from gcpt
df = data[data["Status"].isin(["operating", "mothballed"])]

## Planned retire buckets

In [14]:
#Put planned retire ages into buckets using np.select

# Define conditions
bins = [
    (df['Planned retirement'] >= current_year) & (df['Planned retirement'] <= 2030),  # retire by 2030
    (df['Planned retirement'] > 2030) & (df['Planned retirement'] <= 2040),  # retire by 2040
    (df['Planned retirement'] > 2040) & (df['Planned retirement'] <= 2050),
    #(df['Planned retirement'] > 2040), # retire after 2040
    pd.isna(df['Planned retirement']) | (df['Planned retirement'] > 2050) # NaN values OR retiring after 2050
    #pd.isna(df['Planned retirement'])  # NaN values
]

# Define corresponding labels
labels = ['Retiring by 2030', 'Retiring by 2040', 'Retiring after 2040', 'Remaining']

# Create new column using np.select
df['Planned retire ages'] = np.select(bins, labels, default='Remaining')

# group
grouped = df.groupby(['Country/Area', 'Planned retire ages'])['Capacity'].sum().unstack(fill_value=0)

# Ensure all categories ('by 2030', 'by 2040', 'by 2050', 'remaining') are present for each country
labels = ['Retiring by 2030', 'Retiring by 2040', 'Retiring by 2050', 'Remaining']
#labels = ['Retiring by 2030', 'Retiring by 2040', 'Retiring after 2040', 'Remaining']

# Reindex to include all categories for each country, filling missing ones with 0
grouped = grouped.reindex(columns=labels, fill_value=0)

# reset the index 
grouped.reset_index(inplace=True)

#turn it into tidy data
tidy_df = pd.melt(grouped, id_vars=['Country/Area'], 
                  value_vars=labels, 
                  var_name='Planned retire ages', 
                  value_name='Capacity')

tidy_df.head()

/var/folders/y3/1cxltp3j6rn27pc3lkybs7hw0000gn/T/ipykernel_54462/2141556324.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Planned retire ages'] = np.select(bins, labels, default='Remaining')


,Country/Area,Planned retire ages,Capacity
0,Argentina,Retiring by 2030,0.375
1,Australia,Retiring by 2030,6.098
2,BRICS,Retiring by 2030,20.607
3,Bangladesh,Retiring by 2030,0.000
4,Bosnia and Herzegovina,Retiring by 2030,0.420


In [ ]:
#add missing countries/regions

# From original df, get the unique country/regions.
gcpt_regions = list(data["Country/Area"].unique())

# From df1, get the country/regions that are in it.
existing_countries_and_regions = list(tidy_df["Country/Area"].unique())

# Get the country/regions that are in original but not in df1.
v2 = list(set(gcpt_regions) - set(existing_countries_and_regions))
missing_countries_and_regions = [country_in_region for country_in_region in gcpt_regions if country_in_region not in existing_countries_and_regions]

for country in missing_countries_and_regions:
    for label in labels:
        idx = tidy_df.shape[0]
        tidy_df.loc[idx, "Country/Area"] = country
        tidy_df.loc[idx, "Planned retire ages"] = label
        tidy_df.loc[idx, "Capacity"] = 0
        
tidy_df.reset_index(inplace=True, drop=True)
#tidy_df.head()

In [16]:
tidy_df.head()

,Country/Area,Planned retire ages,Capacity
0,Argentina,Retiring by 2030,0.375
1,Australia,Retiring by 2030,6.098
2,BRICS,Retiring by 2030,20.607
3,Bangladesh,Retiring by 2030,0.000
4,Bosnia and Herzegovina,Retiring by 2030,0.420


In [17]:
#Order the planned retire age column

# Step 1: Define the custom category order - for some reason dashboard flips the order you want
#retirement_order = ['Retiring by 2030', 'Retiring by 2040', 'Retiring by 2050', 'Remaining']
retirement_order = ['Remaining', 'Retiring by 2050', 'Retiring by 2040', 'Retiring by 2030']
#retirement_order = ['Remaining', 'Retiring after 2040', 'Retiring by 2040', 'Retiring by 2030']

# Step 2: Convert 'RetirementStatus' to a categorical type with the specified order
tidy_df['Planned retire ages'] = pd.Categorical(tidy_df['Planned retire ages'], categories=retirement_order, ordered=True)

# Step 3: Sort the DataFrame by 'RetirementStatus'
df_sorted = tidy_df.sort_values('Planned retire ages')

df_sorted = df_sorted.sort_values(["Country/Area", "Planned retire ages"])
df_sorted.rename({"Country/Area": "Country"}, axis=1, inplace=True)
#print(df_sorted)

#write file
df_sorted.to_excel(
    os.path.join(path_to_data_folder, 'planned_retire_groupings.xlsx'), 
    index=False
)

## Past and planned retirements

In [48]:
# Set min and max years
min_year = 2000
max_year = current_year + 1

years_to_add = list(range(min_year, max_year))

In [49]:
#retirements

retired = data[data["Status"].isin(["retired"])].groupby(["Country/Area", "Retired year"]).agg({"Capacity": "sum"}).reset_index()

#retired.rename({"Retired year": "Year", "Capacity": "Retired capacity"}, axis=1, inplace=True)
retired.rename({"Retired year": "Year"}, axis=1, inplace=True)

#retired["Status"] = "Retired"

retired.reset_index(inplace=True, drop=True)

In [50]:
all_countries_and_regions = list(data["Country/Area"].unique())

# From retired, get the country/regions that are in it.
existing_countries_and_regions = list(retired["Country/Area"].unique())

# Get the country/regions that are in original but not in df1.
v2 = list(set(all_countries_and_regions) - set(existing_countries_and_regions))
missing_countries_and_regions = [country_in_region for country_in_region in all_countries_and_regions if country_in_region not in existing_countries_and_regions]

for country in missing_countries_and_regions:
    for year in years_to_add:
        idx = retired.shape[0]
        retired.loc[idx, "Country/Area"] = country
        retired.loc[idx, "Year"] = year
        retired.loc[idx, "Capacity"] = 0
        retired.loc[idx, "Status"] = "Retired"

retired = retired.sort_values(["Country/Area", "Year"])
retired.reset_index(inplace=True, drop=True)

In [51]:
#planned retire
data2 = data.groupby(["Country/Area", "Planned retirement"]).agg({"Capacity": "sum"}).reset_index()

planned_retired = data2[(data2["Planned retirement"]>= datetime.now().year) & (data2["Planned retirement"]<= 2040)]
planned_retired.rename({"Planned retirement": "Year"}, axis=1, inplace=True)

planned_retired["Status"] = "Planned retire"

planned_retired.reset_index(inplace=True, drop=True)

/var/folders/y3/1cxltp3j6rn27pc3lkybs7hw0000gn/T/ipykernel_54462/1806443655.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  planned_retired.rename({"Planned retirement": "Year"}, axis=1, inplace=True)
/var/folders/y3/1cxltp3j6rn27pc3lkybs7hw0000gn/T/ipykernel_54462/1806443655.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  planned_retired["Status"] = "Planned retire"


In [52]:
#add missing countries/regions
all_countries_and_regions = list(data["Country/Area"].unique())

# From planned_retired, get the country/regions that are in it.
existing_countries_and_regions = list(planned_retired["Country/Area"].unique())

# Get the country/regions that are in original but not in df1.
v2 = list(set(all_countries_and_regions) - set(existing_countries_and_regions))
missing_countries_and_regions = [country_in_region for country_in_region in all_countries_and_regions if country_in_region not in existing_countries_and_regions]

# Set min and max years
min_year = current_year
max_year = 2040 + 1

years_to_add = list(range(min_year, max_year))

for country in missing_countries_and_regions:
    for year in years_to_add:
        idx = planned_retired.shape[0]
        planned_retired.loc[idx, "Country/Area"] = country
        planned_retired.loc[idx, "Year"] = year
        planned_retired.loc[idx, "Capacity"] = 0
        planned_retired.loc[idx, "Status"] = "Planned_retire"

planned_retired = planned_retired.sort_values(["Country/Area", "Year"])
planned_retired.reset_index(inplace=True, drop=True)

/var/folders/y3/1cxltp3j6rn27pc3lkybs7hw0000gn/T/ipykernel_54462/1080285140.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  planned_retired.loc[idx, "Country/Area"] = country


In [53]:
all_retired = pd.concat([retired, planned_retired])
#all_retired = all_retired.reset_index(inplace=True, drop=True)
all_retired = all_retired.sort_values(["Country/Area", "Year"], ascending=True).reset_index(drop=True)
all_retired.rename({"Country/Area": "Country"}, axis=1, inplace=True)

# write file
all_retired.to_excel(
    os.path.join(path_to_data_folder, 'past_and_planned_retirements.xlsx'), 
    index=False
)

## Gap in ambition - average annual retirements

In [64]:
# Filter capacity_status data for rows with Year = 2024
capacity_2024 = capacity_data[capacity_data['Year'] == "H1 2024"][['Country/Area', 'Operating']]

# Merge with OECD data based on 'Country/Area'
df = pd.merge(oecd_data, capacity_2024, on='Country/Area', how='left')

# Save the updated OECD data to a new file
# df.to_excel('/path/to/Updated_OECD.xlsx', index=False)

#df.head()

In [65]:
#for each Country/Area I want to get the coal phaseout date, subtract it from the current year, and save it to a variable called remaining_years

# Calculate the remaining years and store them in a new column
df['remaining_years'] = df['1.5 pathway'] - current_year + 1

# get remaining years
df['1.5C annual retirements needed'] = df['Operating'] / df['remaining_years']

In [66]:

#retired_df = pd.read_excel('output/Past and planned retire.xlsx')

# Merge the two dataframes on 'Country/Area'
merged_df = pd.merge(planned_retired, df, on='Country/Area', how='left')

# Filter the rows where 'Year' is between 'current_year' and 'Coal phaseout year'
merged_df_filtered = merged_df[(merged_df['Year'] >= current_year + 1) & (merged_df['Year'] <= merged_df['1.5 pathway'])]

# Sum the capacity for each country/area
retirement_sums = merged_df_filtered.groupby('Country/Area')['Capacity'].sum()

# Add the 'retirement_sum' column to df, matching by 'Country/Area'
df['retirement_sum'] = df['Country/Area'].map(retirement_sums)

#get annual retirements planned
df['Annual retirements planned'] = df['retirement_sum'] / df['remaining_years']

# Display the updated df
df.head()

,Country/Area,1.5 pathway,Operating,remaining_years,1.5C annual retirements needed,retirement_sum,Annual retirements planned
0,Global,2040,2125.5271,17,125.031006,301.8211,17.754182
1,G7,2030,295.9937,7,42.284814,105.3884,15.055486
2,G20,2040,1968.1992,17,115.776424,265.2461,15.602712
3,EU27,2030,97.4918,7,13.927400,46.5096,6.644229
4,OECD,2030,442.9242,7,63.274886,145.3945,20.770643


In [67]:
# Melt the dataframe to reshape it into a long format
melted_df = pd.melt(df, id_vars=['Country/Area'], value_vars=['1.5C annual retirements needed', 'Annual retirements planned'],
                    var_name='Retirements', value_name='Capacity')

# Display the result
melted_df.rename({"Country/Area": "Country"}, axis=1, inplace=True)

#add NZ pathway year
melted_df_with_nz_pathway_year = pd.merge(melted_df, df[["Country/Area", "1.5 pathway"]], left_on="Country", right_on="Country/Area", how="left").drop("Country/Area", axis=1)

In [68]:
for idx, row in melted_df_with_nz_pathway_year.iterrows():
    retirements = row["Retirements"]
    nz_year = row["1.5 pathway"]
    
    #retirements += f" (2025 - {nz_year})"
    retirements += f" to {nz_year}"
    
    melted_df_with_nz_pathway_year.loc[idx, "Retirements"] = retirements
    
melted_df_with_nz_pathway_year.drop("1.5 pathway", axis=1, inplace=True)
    

In [69]:
melted_df_with_nz_pathway_year

,Country,Retirements,Capacity
0,Global,1.5C annual retirements needed to 2040,125.031006
1,G7,1.5C annual retirements needed to 2030,42.284814
2,G20,1.5C annual retirements needed to 2040,115.776424
3,EU27,1.5C annual retirements needed to 2030,13.927400
4,OECD,1.5C annual retirements needed to 2030,63.274886
...,...,...,...
225,Uzbekistan,Annual retirements planned to 2040,0.000000
226,Venezuela,Annual retirements planned to 2040,0.000000
227,Vietnam,Annual retirements planned to 2040,0.000000
228,Zambia,Annual retirements planned to 2040,0.000000


In [70]:
melted_df_with_nz_pathway_year.to_excel(
    os.path.join(path_to_data_folder, 'gap_in_ambition.xlsx'),
    index=False
)

## Data ticker

In [33]:
# Filter capacity_status data for rows with Year = 2024
capacity_2024 = capacity_data[capacity_data['Year'] == "H1 2024"][['Country/Area', 'Operating']]

# Merge with OECD data based on 'Country/Area'
oecd_df = pd.merge(oecd_data, phaseout_data, on='Country/Area', how='left')
data_ticker = pd.merge(oecd_df, capacity_2024, on='Country/Area', how='left')
data_ticker  = data_ticker.rename(columns={'Operating': 'Total operating capacity (GW)', 'Pledged coal': 'Pledged coal phaseout year', '1.5 pathway': '1.5C pathway phaseout year'})

data_ticker.rename({"Country/Area": "Country"}, axis=1, inplace=True)
#data_ticker.head()
data_ticker.to_excel(
    os.path.join(path_to_data_folder, 'data_ticker.xlsx'),
    index=False
)

## Text config file

In [25]:
#merge phaseout data with df from average annual retirements
df2 = pd.merge(df, phaseout_data, on='Country/Area', how='left')

#get percentage of capacity planned to retire
df2['percent_retired'] = ((df2['retirement_sum'] / df2['Operating']) * 100)

#round the number columns based on value
def custom_round(value):
    # Perform rounding based on the value
    if value >= 10:
        rounded_value = np.round(value, 0)  # Round to 0 decimals for values >= 10
    elif 1 <= value < 10:
        rounded_value = np.round(value, 1)  # Round to 1 decimal place for values between 1 and 10
    else:
        rounded_value = np.round(value, 2)  # Round to 2 decimal places for values < 1
    
    # Return the rounded value as a string with ".0" removed if it's a whole number
    return str(rounded_value).replace(".0", "")

# Apply the custom rounding function to the columns
df2['percent_retired'] = df2['percent_retired'].apply(custom_round)
df2['Operating'] = df2['Operating'].apply(custom_round)
df2['retirement_sum'] = df2['retirement_sum'].apply(custom_round)
df2['Pledged coal'] = df2['Pledged coal'].apply(custom_round)

In [ ]:
# Define region-specific summaries for both the intro text and pledged coal text
region_mapping = {
    "Global": {
        "intro": "According to the IEA's Net Zero scenario, advanced economies should phase out all unabated coal power by 2030 and the rest of the world by 2040.",
        "pledge": "The world as a whole has not yet pledged to phase out coal power on a specific timeline, although several individual countries and regions have."
    },
    "G7": {
        "intro": "According to the IEA's Net Zero scenario, G7 nations as advanced economies should phase out all unabated coal power by 2030.",
        "pledge": "The G7 have committed to phasing out coal power by 2035."
    },
    "G20": {
        "intro": "According to the IEA's Net Zero scenario, advanced economies should phase out all unabated coal power by 2030 and the rest of the world by 2040. Several G20 countries are considered advanced economies.",
        "pledge": "The G20 as a group has not committed to phasing out coal power, although some individual G20 countries have."
    },
    "OECD": {
        "intro": "According to the IEA's Net Zero scenario, advanced economies should phase out all unabated coal power by 2030 and the rest of the world by 2040. Most countries within the OECD are considered advanced economies.",
        "pledge": "The OECD as a group has not committed to phasing out coal power, although some individual OECD countries have."
    },
    "BRICS": {
        "intro": "According to the IEA's Net Zero scenario, BRICS countries should phase out all unabated coal power by 2040.",
        "pledge": "BRICS countries have not made a pledge to phase out coal power."
    }
}

# Function to get region-specific text for intro and pledge
def get_region_text(country_or_region, text_type):
    # Fetch the region-specific text based on the country_or_region and text_type (either "intro" or "pledge")
    return region_mapping.get(country_or_region, {}).get(text_type, None)

# Iterate through countries and regions
for country in df2['Country/Area'].unique():
    row = df2[df2['Country/Area'] == country].iloc[0]
    
    # Get region-specific intro text
    region_intro_text = get_region_text(row['Country/Area'], 'intro')
    
    if region_intro_text:
        # If it's a region, use the region-specific intro text
        intro_text = region_intro_text
    else:
        # If it's an individual country, use the general intro text
        intro_text = f"According to the IEA's Net Zero scenario, {row['Country/Area']} should phase out all unabated coal power by {row['1.5 pathway']}."
    
    # Operating text
    if (row['Operating'] == 0) or (row['Operating'] == "0"):
        operating_text = f"{row['Country/Area']} has no operating coal power."
    elif row['Country/Area'] == "Global":
        operating_text =f"As of January 2025, there are plans to retire {row['percent_retired']}% ({row['retirement_sum']} GW) of the world's {row['Operating']} GW operating coal fleet by {row['1.5 pathway']}."
    else:
        operating_text = f"As of January 2025, {row['Country/Area']} plans to retire {row['percent_retired']}% ({row['retirement_sum']} GW) of its {row['Operating']} GW operating coal fleet by {row['1.5 pathway']}."
    
    # Get region-specific pledged coal text
    region_pledged_text = get_region_text(row['Country/Area'], 'pledge')
    
    if (pd.isna(row['Pledged coal'])) or (row['Pledged coal'] == "nan"):
        if region_pledged_text:
            pledged_text = region_pledged_text
        else:
            pledged_text = f"{row['Country/Area']} has not pledged to phase out coal power."
    else:
        pledged_text = f"{row['Country/Area']} has pledged to phase out coal by {row['Pledged coal']}."

    # Combine everything into the overall summary
    overall_summary = f"{intro_text} {operating_text} {pledged_text}"
    
    # Update the DataFrame with the overall summary
    df2.loc[df2['Country/Area'] == country, 'overall_summary'] = overall_summary

# Output the result
print(df2[['Country/Area', 'overall_summary']])

    Country/Area                                    overall_summary
0         Global  According to the IEA's Net Zero scenario, adva...
1             G7  According to the IEA's Net Zero scenario, G7 n...
2            G20  According to the IEA's Net Zero scenario, adva...
3           EU27  According to the IEA's Net Zero scenario, EU27...
4           OECD  According to the IEA's Net Zero scenario, adva...
..           ...                                                ...
110   Uzbekistan  According to the IEA's Net Zero scenario, Uzbe...
111    Venezuela  According to the IEA's Net Zero scenario, Vene...
112      Vietnam  According to the IEA's Net Zero scenario, Viet...
113       Zambia  According to the IEA's Net Zero scenario, Zamb...
114     Zimbabwe  According to the IEA's Net Zero scenario, Zimb...

[115 rows x 2 columns]


In [77]:
final = df2[['Country/Area', 'overall_summary']]
final.rename({"Country/Area": "Country"}, axis=1, inplace=True)
final.to_excel(
    os.path.join(path_to_data_folder, 'text_config.xlsx'),
    index=False
)

/var/folders/2s/38kg81v526j7qcgd8gdq8jd00000gn/T/ipykernel_76656/3421105229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.rename({"Country/Area": "Country"}, axis=1, inplace=True)


# Updating text-config.json

In [78]:
# Read in the json file as a dictionary.
with open("../public/assets/text-config.json", "r") as f:
    text_config = json.load(f)

In [79]:
# Make changes to the dictionary.

dropdown = []

for idx, row in final.iterrows():
    country = row["Country"]
    summary = row["overall_summary"]
    
    dictionary = {}
    
    dictionary["Country"] = country
    dictionary["overall_summary"] = summary

    dropdown.append(dictionary)


text_config["dropdown"] = dropdown

In [80]:
# Write the dictionary to the previous location.
with open("/Users/nicholasabad/Desktop/workspace/CodeOp/daft04/testing-repository/Phaseout-dashboard/public/assets/text-config.json", "w") as f:
    json.dump(
        text_config,
        f,
        indent=2
    )
